Ok, so at the end of the last chapter, we found that the functional test's attempt to write to the table in our HTML to-do list was not saving the input.  Harry says that you need more than just Selenium to do that; you need to specify how to get the input into a database backing up the page.  You could use HTML5 or JavaScript, but we start with a simple HTML POST request.

The way this is done is actually to write it into our `home.html` template file, with an `input` possessing a `name` attribute, and wrapping that in a `<form method="POST">` tag.  Furthermore, this tag goes around your previously-entered `input` item with the `placeholder` attribute saying `"Enter a to-do item"`:

In [2]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input id="id_new_item" placeholder="Enter a to-do item" />
        </form>
        <table id="id_list_table">
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


In [4]:
# !python functional_tests.py

Yields

`selenium.common.exceptions.NoSuchElementException: Message: Unable to locate element: [id="id_list_table"]`

as anticipated by the text.

He suggests that, when confronted with an error that's hard to interpret, like this one, there are a few common steps to consider trying:

* Add `print` statements to get more info about the state (or, presumably, execute with debugging turned on in VS Code or other IDE)
* Try to write more context into your Error statements, to reflect the current state of the interpreter
* Try firing up your app server and just check the page yourself (i.e., not within a functional test)
* Use `time.sleep` to slow down the execution, giving you more time to visualize what's going on with the site

We actually already have the `time.sleep` option implemented in the functional test, so he suggests just extending it from 1 second as a quick sanity check.

In [6]:
%%writefile functional_tests.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(60)

        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertTrue(
            any(row.text == '1: Buy peacock feathers' for row in rows),
            "New to-do item did not appear in table"
        )

        # There is still a text box inviting her to add another item.
        # Shee enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        self.fail('Finish the test!')

        # The page updates again, and now shows both items on her list
        # ...

if __name__ == '__main__':
    unittest.main(warnings='ignore')


Overwriting functional_tests.py


In [8]:
!python functional_tests.py

E
ERROR: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 40, in test_can_start_a_list_and_retrieve_it_later
    table = self.browser.find_element_by_id('id_list_table')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 360, in find_element_by_id
    return self.find_element(by=By.ID, value=id_)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 978, in find_element
    'value': value})['value']
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\selenium\webdriver\remote\errorhandler.py", li

Ok, doing that lets you see the page before the test checks and closes the browser.  Major text output is:

<blockquote><p>CSRF verification failed. Request aborted.</p><p>You are seeing this message because this site requires a CSRF cookie when submitting forms. This cookie is required for security reasons, to ensure that your browser is not being hijacked by third parties.</p><p>If you have configured your browser to disable cookies, please re-enable them, at least for this site, or for 'same-origin' requests.</p></blockquote>

That's a *bit* more context: something about cookies.  The text at this point has a box indicating that CSRF stands for "*Cross-Site Request Forgery*", a form of exploit.  There's no real way to anticipate what the heck it's about and how to deal with it without a deep dive into the web, but it turns out that there's a specific way that [Django handles CSRF](https://docs.djangoproject.com/en/4.0/ref/csrf/) that involves just inserting the template magic

`{% csrf_token %}`

into the POST request form, right after your `input` tag:

In [12]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


In [13]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 44, in test_can_start_a_list_and_retrieve_it_later
    "New to-do item did not appear in table"
AssertionError: False is not true : New to-do item did not appear in table

----------------------------------------------------------------------
Ran 1 test in 66.777s

FAILED (failures=1)


So adding the `csrf_token` part was enough to get past the original error.  When I tried entering the expected text manually once the site launched, though, it still doesn't write it to any table.  Harry says that's because we haven't instructed Django on how to process any POST requests yet.  He says the way to do that is to write an `action=attribute` into the template; first, though, you add a unit test to `lists/tests.py` to anticipate this, and some conditional logic into the `home_page` func in the `lists/views.py` file.  So it's kind of a(nother) rabbit hole of HTML and web dev with which I'm unfamiliar and therefore complicates greatly the gist of understanding testing.  So instead of reading up on it and going off on a huge tangent about what all of this stuff means, I'll implement it verbatim from the text, and try to stick to the idea of understanding what it means in the context of TDD: how you'd anticipate the changes to the app code and write corresponding unit & functional tests.

First, we create a `test_can_save_a_POST_request` func for the unit tests file:

In [34]:
%%writefile lists/tests.py
from django.test import TestCase

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertIn('A new list item', response.content.decode())

Overwriting lists/tests.py


Then, the conditional logic for the `home_page` func in our `views` file:

In [32]:
%%writefile lists/views.py
from django.shortcuts import render
from django.http import HttpResponse

def home_page(request):
    if request.method == 'POST':
        return HttpResponse(request.POST['item_text'])
    return render(request, 'home.html')


Overwriting lists/views.py


In [35]:
# !python functional_tests.py
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


..
----------------------------------------------------------------------
Ran 2 tests in 0.026s

OK


All right, but then we turn around and say that's just to get the tests to pass, and not really the way you want to implement it after all.  ~~And, at present, it's not even passing in my hands.~~  (I had missed the detail that the change was only meant to get the **unit tests** to pass, not necessarily the functional tests... we're back in accord with the text.)  So move on and check out the more authoritative way forward, as revealed later in the chapter.

It involves leveraging more of the Django syntax for incorporating template text based on Python variables, which we've already seen with the whole `csrf_token` magic, earlier.  Except that if you're not using Django's magic, but instead native Python variables, apparently the syntax is to include in *double* curly braces:

In [63]:
%%writefile lists/templates/home.html
<html>
    <head>
        <title>To-Do lists</title>
    </head>
    <body>
        <h1>Your To-Do list</h1>
        <form method="POST">
            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />
            {% csrf_token %}
        </form>
        <table id="id_list_table">
            <tr><td>{{ new_item_text }}</td></tr>
        </table>
    </body>
</html>


Overwriting lists/templates/home.html


With the explanation that `new_item_text` in the template is meant to distinguish it from the `item_text` specified in the `views.py` file in order to force us to realize that the one does not become the other automatically, but only via the action of interpreting the template via a view.

Then we modify the unit tests file again:

In [40]:
%%writefile lists/tests.py
from django.test import TestCase

class HomePageTest(TestCase):

    def test_home_page_returns_correct_html(self):
        response = self.client.get('/')
        self.assertTemplateUsed(response, 'home.html')

    def test_can_save_a_POST_request(self):
        response = self.client.post('/', data={'item_text': 'A new list item'})
        self.assertIn('A new list item', response.content.decode())
        self.assertTemplateUsed(response, 'home.html')


Overwriting lists/tests.py


In [41]:
# !python functional_tests.py
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


F.
FAIL: test_can_save_a_POST_request (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 12, in test_can_save_a_POST_request
    self.assertTemplateUsed(response, 'home.html')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\testcases.py", line 578, in assertTemplateUsed
    self.fail(msg_prefix + "No templates used to render the response")
AssertionError: No templates used to render the response

----------------------------------------------------------------------
Ran 2 tests in 0.024s

FAILED (failures=1)
E
ERROR: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 40, in test_can_start_a_list_and_retrieve_it_later
    table = self.browser.

Ok, unsure about the second error, but the text anticipates the first one, as the unit test no longer passing due to our original, overly-simplistic code written into the views file.  So we update that, and Harry says that the `django.shortcuts.render` func accepts an optional *third* arg, which should be a dict mapping the template names to specific values.

In [43]:
%%writefile lists/views.py
from django.shortcuts import render
from django.http import HttpResponse

def home_page(request):
    return render(request, 'home.html', {
        'new_item_text': request.POST['item_text']
    })


Overwriting lists/views.py


In [44]:
!python manage.py test

Creating test database for alias 'default'...
System check identified no issues (0 silenced).
Destroying test database for alias 'default'...


FE
ERROR: test_home_page_returns_correct_html (lists.tests.HomePageTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\utils\datastructures.py", line 83, in __getitem__
    list_ = super(MultiValueDict, self).__getitem__(key)
KeyError: 'item_text'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Data\projects\test_driven_development\lists\tests.py", line 6, in test_home_page_returns_correct_html
    response = self.client.get('/')
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 536, in get
    **extra)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\site-packages\django\test\client.py", line 340, in get
    return self.generic('GET', path, secure=secure, **r)
  File "C:\Users\DCM0303\Miniconda3\envs\test_dev_book\lib\s

# Unexpected Failure

Ok, so now we're confronted with an error that the author intended as an unexpected one (again, since I don't know a damn thing about Django/HTML, they're *all* unexpected to me); the code doesn't know what to do now when it *doesn't* receive a POST request.  His point is that if you don't write tests (or only write ones to catch the things you *expect* to be frequent failure cases), you may not anticipate the weird stuff that can happen due to your lack of imagination.

The fix is pretty simple; modify the third arg we just passed to `render` in the views file.  Specifically, instead of direct bracket-based indexing of our Python dict, use the `get` method, which in turn accepts a second arg as what to return should the key not be present in your dict (we want that default value to be an empty string, here):

In [55]:
%%writefile lists/views.py
from django.shortcuts import render
from django.http import HttpResponse

def home_page(request):
    return render(request, 'home.html', {
        'new_item_text': request.POST.get('item_text', '')
    })


Overwriting lists/views.py


In [56]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 44, in test_can_start_a_list_and_retrieve_it_later
    "New to-do item did not appear in table"
AssertionError: False is not true : New to-do item did not appear in table

----------------------------------------------------------------------
Ran 1 test in 66.687s

FAILED (failures=1)


Also in accord with the text up to this point.  The author suggests further modifying the error statement encountered to be even more informative (by printing the content of the variable that it's checking):

In [61]:
%%writefile functional_tests.py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import unittest

class NewVisitorTest(unittest.TestCase):

    def setUp(self):
        self.browser = webdriver.Firefox()

    def tearDown(self):
        self.browser.quit()

    def test_can_start_a_list_and_retrieve_it_later(self):
        # Edith has heard about a cool new online to-do app.  She goes
        # to check out its homepage
        self.browser.get('http://localhost:8000')

        # She notices the page title and header mention to-do lists
        self.assertIn('To-Do', self.browser.title)
        header_text = self.browser.find_element_by_tag_name('h1').text
        self.assertIn('To-Do', header_text)

        # She is invited to enter a to-do item straight away
        inputbox = self.browser.find_element_by_id('id_new_item')
        self.assertEqual(
            inputbox.get_attribute('placeholder'),
            'Enter a to-do item'
        )

        # She types "Buy peacock feathers" into a text box 
        # (Edith's hobby is tying fly-fishing lures)
        inputbox.send_keys('Buy peacock feathers')

        # When she hits enter, the page updates, and now the page lists
        # "1: Buy peacock feathers" as an item in a to-do list table
        inputbox.send_keys(Keys.ENTER)
        time.sleep(30)

        table = self.browser.find_element_by_id('id_list_table')
        rows = table.find_elements_by_tag_name('tr')
        self.assertIn('1: Buy peacock feathers', [row.text for row in rows])
        # self.assertTrue(
        #     any(row.text == '1: Buy peacock feathers' for row in rows),
        #     f"New to-do item did not appear in table.  Contents were:\n{table.text}"
        # )

        # There is still a text box inviting her to add another item.
        # Shee enters "Use peacock feathers to make a fly"
        # (Edith is very methodical)
        self.fail('Finish the test!')

        # The page updates again, and now shows both items on her list
        # ...

if __name__ == '__main__':
    unittest.main(warnings='ignore')


Overwriting functional_tests.py


In [64]:
!python functional_tests.py

F
FAIL: test_can_start_a_list_and_retrieve_it_later (__main__.NewVisitorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "functional_tests.py", line 42, in test_can_start_a_list_and_retrieve_it_later
    self.assertIn('1: Buy peacock feathers', [row.text for row in rows])
AssertionError: '1: Buy peacock feathers' not found in ['1. Buy peacock feathers']

----------------------------------------------------------------------
Ran 1 test in 36.857s

FAILED (failures=1)


Ok, one issue that I encountered was that I hadn't copied the `input` tag over from the text correctly; it needed to be

            <input name="item_text" id="id_new_item" placeholder="Enter a to-do item" />

When I added the `<form method="POST">` earlier in the chapter, I had failed to add that `name="item_text"` attribute to the `input` tag within it.

Anyways, the author tried modifying the `test_can_start_a_list_and_retrieve_it_later` functional test at first by forcing his error statement to dump the contents of whatever the user had input, but then adds a note that someone else pointed out that an `assertIn` call there would be simpler than `assertTrue`.  And that using `assertIn` forces the error statement to include the variable you're inspecting whenever it fails, simplifying the test.  His overall point is that whenever you think you've found some clever solution for something, you're probably in fact just making the code overly complicated, and should search your docs/API/library for another method or entry point that's more directly relevant for the thing you're trying to do.

And then, the final lesson from this section is just that the test now fails for trivial reasons: the thing you're trying to ensure a user enters doesn't *exactly* match the string that the user may choose to input at a given time.  No general advice about avoiding this, though; seems dumb to hard-code a test that's quite so picky about formatting of the input string.  Maybe a regex would be more appropriate?